In [1]:
%run setup.ipynb

### YouTube Transcript Loader
You can get the transcript from any YouTube link. First, you have to install the given libraries. If you want to keep the video information like title, and description in the documents, then keep add_video_info=True Otherwise keep False.

In [5]:
%pip install --upgrade -qq  youtube-transcript-api
%pip install --upgrade -qq  pytube

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install --upgrade pytube

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=LAfrShnpVIk",
    add_video_info=False,
)

result = loader.load()
print(f"Without Video Info: {result}\n\n")

Without Video Info: [Document(metadata={'source': 'LAfrShnpVIk'}, page_content="when using chat GPT you most probably have encountered responses like I'm sorry but as of my last knowledge update in January 2022 Etc or even responses that are not true at all this is where rag comes into play and says let me help you by injecting more knowledge or content into your interactions with an llm and help it answer the unknown and upcoming questions we hear llms prompts and rag Everywhere by now I think most of us know what an llm and the prompt is but did you know that right now rag is just as important as both of these and Powers most applications you may use involving a chatbot I recently did a poll on the learn AI together Discord Community to find out if people had already studied created or used rag applications and most voted to understand what rag is used for rag is as important as your coursebook for success in a class so understanding what it is is highly relevant in AI an llm or a la

In [12]:
# 🔧 SSL Certificate Fix for YouTube API Access
# This addresses the common SSL certificate verification error when accessing YouTube

import ssl
import urllib.request

def setup_ssl_bypass():
    """Setup SSL context to bypass certificate verification for YouTube API access."""
    try:
        # Create unverified SSL context
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        # Legacy Python that doesn't verify HTTPS certificates by default
        pass
    else:
        # Handle target environment that doesn't support HTTPS verification
        ssl._create_default_https_context = _create_unverified_https_context
    
    print("✅ SSL certificate bypass configured for YouTube API access")

# Apply the SSL bypass
setup_ssl_bypass()

# Additional SSL configuration for pytube
import os
os.environ['PYTHONHTTPSVERIFY'] = '0'


✅ SSL certificate bypass configured for YouTube API access


In [13]:
# Enhanced YouTube loader with SSL error handling
from urllib.error import URLError
import ssl

def load_youtube_with_info(url, max_retries=3):
    """Load YouTube video with info, handling SSL errors gracefully."""
    
    for attempt in range(max_retries):
        try:
            print(f"🔄 Attempt {attempt + 1}/{max_retries} - Loading YouTube video with info...")
            
            loader = YoutubeLoader.from_youtube_url(
                url,
                add_video_info=True,
            )
            result = loader.load()
            print("✅ Successfully loaded YouTube video with metadata!")
            return result
            
        except URLError as e:
            if "CERTIFICATE_VERIFY_FAILED" in str(e):
                print(f"⚠️  SSL Certificate error on attempt {attempt + 1}")
                if attempt < max_retries - 1:
                    print("🔧 Applying additional SSL bypass...")
                    # Apply more aggressive SSL bypass
                    import ssl
                    ssl._create_default_https_context = ssl._create_unverified_context
                    continue
                else:
                    print("❌ SSL issues persist. Falling back to transcript-only mode...")
                    # Fallback to transcript only
                    loader = YoutubeLoader.from_youtube_url(url, add_video_info=False)
                    result = loader.load()
                    print("✅ Successfully loaded transcript (without video metadata)")
                    return result
            else:
                print(f"❌ Other URL error: {e}")
                raise
        except Exception as e:
            print(f"❌ Unexpected error on attempt {attempt + 1}: {e}")
            if attempt == max_retries - 1:
                print("🔄 Falling back to transcript-only mode...")
                loader = YoutubeLoader.from_youtube_url(url, add_video_info=False)
                result = loader.load()
                print("✅ Successfully loaded transcript (without video metadata)")
                return result

# Test the enhanced loader
try:
    result = load_youtube_with_info("https://www.youtube.com/watch?v=ZL-cwYRMPjI")
    print(f"\n📊 Results Summary:")
    print(f"   - Number of documents: {len(result)}")
    print(f"   - Document metadata: {result[0].metadata}")
    print(f"   - Content preview: {result[0].page_content[:200]}...")
except Exception as e:
    print(f"❌ Final error: {e}")
    print("💡 Try restarting the kernel or use transcript-only mode")

🔄 Attempt 1/3 - Loading YouTube video with info...
❌ Other URL error: HTTP Error 400: Bad Request
❌ Final error: HTTP Error 400: Bad Request
💡 Try restarting the kernel or use transcript-only mode


### 🔧 SSL Certificate Troubleshooting for YouTube Loader

If you encounter SSL certificate verification errors when using `YoutubeLoader` with `add_video_info=True`, here's what's happening and how to fix it:

#### **Problem:**
```
URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate>
```

#### **Root Cause:**
- The `pytube` library (used internally by `YoutubeLoader`) needs to access YouTube's API to fetch video metadata
- macOS and some environments have SSL certificate verification issues with Python's urllib
- This only affects video metadata fetching, not transcript loading

#### **Solutions Applied:**

1. **SSL Certificate Bypass:** The cell above configures Python to bypass SSL verification for HTTPS requests
2. **Environment Variable:** Sets `PYTHONHTTPSVERIFY=0` to disable SSL verification
3. **Graceful Fallback:** Enhanced loader automatically falls back to transcript-only mode if SSL issues persist

#### **Alternative Solutions:**

If you still encounter issues, try these manual fixes:

1. **Use transcript-only mode:**
   ```python
   loader = YoutubeLoader.from_youtube_url(url, add_video_info=False)
   ```

2. **Install certificates (macOS):**
   ```bash
   /Applications/Python\ 3.11/Install\ Certificates.command
   ```

3. **Update certificates:**
   ```bash
   pip install --upgrade certifi
   ```

4. **Restart kernel:** Sometimes the SSL context needs a fresh start

#### **Notes:**
- SSL bypass is **only for development/testing** environments
- For production, prefer proper certificate configuration
- The enhanced loader above handles these issues automatically


If you want to format the Transcripts, you can do it through the TranscriptFormat class. Here I have created the documents where each document will contain the transcript of 10-second contexts from the video. You can customize it in your desired format.

In [14]:
# 📑 YouTube Transcript with Custom Formatting
from langchain_community.document_loaders.youtube import TranscriptFormat

def load_youtube_transcript_chunks(url, chunk_size_seconds=10):
    """Load YouTube transcript in formatted chunks."""
    try:
        print(f"📺 Loading YouTube transcript in {chunk_size_seconds}-second chunks...")
        
        loader = YoutubeLoader.from_youtube_url(
            url,
            add_video_info=False,  # Use False to avoid SSL issues
            transcript_format=TranscriptFormat.CHUNKS,
            chunk_size_seconds=chunk_size_seconds,
        )
        
        documents = loader.load()
        print(f"✅ Successfully loaded {len(documents)} transcript chunks")
        
        return documents
        
    except Exception as e:
        print(f"❌ Error loading transcript: {e}")
        return []

# Load and display formatted transcript
documents = load_youtube_transcript_chunks("https://www.youtube.com/watch?v=TKCMw0utiak")

if documents:
    print(f"\n📊 Transcript Summary:")
    print(f"   - Total chunks: {len(documents)}")
    print(f"   - Sample metadata: {documents[0].metadata}")
    print(f"\n🔤 First few chunks:")
    print("=" * 50)
    
    # Display first 3 chunks
    for i, doc in enumerate(documents[:3]):
        print(f"📄 Chunk {i+1}:")
        print(f"   Content: {doc.page_content}")
        print(f"   Metadata: {doc.metadata}")
        print("-" * 30)
else:
    print("❌ No transcript chunks loaded")

📺 Loading YouTube transcript in 10-second chunks...
✅ Successfully loaded 16 transcript chunks

📊 Transcript Summary:
   - Total chunks: 16
   - Sample metadata: {'source': 'https://www.youtube.com/watch?v=TKCMw0utiak&t=0s', 'start_seconds': 0, 'start_timestamp': '00:00:00'}

🔤 First few chunks:
📄 Chunk 1:
   Content: ♪ Hail to the victors valiant ♪
   Metadata: {'source': 'https://www.youtube.com/watch?v=TKCMw0utiak&t=0s', 'start_seconds': 0, 'start_timestamp': '00:00:00'}
------------------------------
📄 Chunk 2:
   Content: ♪ Hail to the conquering heroes ♪ ♪ Hail, hail to Michigan ♪
   Metadata: {'source': 'https://www.youtube.com/watch?v=TKCMw0utiak&t=10s', 'start_seconds': 10, 'start_timestamp': '00:00:10'}
------------------------------
📄 Chunk 3:
   Content: ♪ The leaders and best ♪
   Metadata: {'source': 'https://www.youtube.com/watch?v=TKCMw0utiak&t=20s', 'start_seconds': 20, 'start_timestamp': '00:00:20'}
------------------------------


In [15]:
# 🧪 Complete YouTube Loader Test Suite
# This cell tests all YouTube loader functionality with proper error handling

def test_youtube_loader_comprehensive():
    """Test all YouTube loader functionality comprehensively."""
    
    test_video_url = "https://www.youtube.com/watch?v=LAfrShnpVIk"
    
    print("🧪 YouTube Loader Comprehensive Test")
    print("=" * 50)
    
    # Test 1: Basic transcript loading (should always work)
    print("\n1️⃣ Testing basic transcript loading...")
    try:
        loader = YoutubeLoader.from_youtube_url(test_video_url, add_video_info=False)
        result = loader.load()
        print(f"✅ Basic transcript: {len(result)} documents loaded")
        print(f"   Content preview: {result[0].page_content[:100]}...")
    except Exception as e:
        print(f"❌ Basic transcript failed: {e}")
    
    # Test 2: Transcript with video info (may fail due to SSL)
    print("\n2️⃣ Testing transcript with video info...")
    try:
        result = load_youtube_with_info(test_video_url)
        print(f"✅ With video info: {len(result)} documents loaded")
        print(f"   Metadata: {result[0].metadata}")
    except Exception as e:
        print(f"❌ With video info failed: {e}")
    
    # Test 3: Chunked transcript
    print("\n3️⃣ Testing chunked transcript...")
    try:
        documents = load_youtube_transcript_chunks(test_video_url, chunk_size_seconds=15)
        if documents:
            print(f"✅ Chunked transcript: {len(documents)} chunks loaded")
            print(f"   First chunk: {documents[0].page_content[:50]}...")
        else:
            print("❌ No chunks loaded")
    except Exception as e:
        print(f"❌ Chunked transcript failed: {e}")
    
    print("\n🏁 Test suite completed!")
    print("💡 If any tests failed, check the troubleshooting guide above")

# Run the comprehensive test
test_youtube_loader_comprehensive()


🧪 YouTube Loader Comprehensive Test

1️⃣ Testing basic transcript loading...
✅ Basic transcript: 1 documents loaded
   Content preview: when using chat GPT you most probably have encountered responses like I'm sorry but as of my last kn...

2️⃣ Testing transcript with video info...
🔄 Attempt 1/3 - Loading YouTube video with info...
❌ Other URL error: HTTP Error 400: Bad Request
❌ With video info failed: HTTP Error 400: Bad Request

3️⃣ Testing chunked transcript...
📺 Loading YouTube transcript in 15-second chunks...
✅ Successfully loaded 39 transcript chunks
✅ Chunked transcript: 39 chunks loaded
   First chunk: when using chat GPT you most probably have encount...

🏁 Test suite completed!
💡 If any tests failed, check the troubleshooting guide above
